In [ ]:
import time
import numpy as np
import random

from IPython.core.debugger import set_trace

import import_ipynb
from placement_cplex import *
from config import *
from placement_ortools import *
from placement_ortools_multiobjective import *

%load_ext autoreload
%autoreload 2

In [ ]:
SIZE_LEARNING_DATASET = 50000
SIZE_TESTING_DATASET = 50

In [ ]:
def create_random_instance(config):
    
    machines = np.arange(config.num_tasksperjob)
    machines = np.tile(machines, (config.num_jobs,1))
    for job in machines:
        np.random.shuffle(job)
        
    durations = np.random.randint(low=config.task_minTime, high=config.task_maxTime, size=(config.num_jobs, config.num_tasksperjob))

    return machines, durations

In [ ]:
def read_instances(config, file_test):
    
    testing_machines = []
    testing_duration = []
    testing_solver = []

    # Testing dataset

    num_lines = len(open(file_test).readlines())

    with open(file_test, "r") as file:

        for _ in range(num_lines // (config.num_jobs+2)):

            NB_JOBS, NB_MACHINES = [int(v) for v in file.readline().split()]
            JOBS = [[int(v) for v in file.readline().split()] for i in range(NB_JOBS)]
            testing_solver.append(int(float(file.readline().split()[1])))

            # Build list of machines. MACHINES[j][s] = id of the machine for the operation s of the job j
            testing_machines.append([[JOBS[j][2 * s] for s in range(NB_MACHINES)] for j in range(NB_JOBS)])

            # Build list of durations. DURATION[j][s] = duration of the operation s of the job j
            testing_duration.append([[JOBS[j][2 * s + 1] for s in range(NB_MACHINES)] for j in range(NB_JOBS)])


    testing_machines = np.array(testing_machines)
    testing_duration = np.array(testing_duration)
    testing_solver = np.array(testing_solver)
    
    return testing_machines, testing_duration

In [ ]:
def create_learning_dataset(config, filePath):
        
    # Save in learning_dataset.csv
    
    for _ in range(SIZE_LEARNING_DATASET):
        
        machines, durations = create_random_instance(config)

        # Open the file with writing permission
        myfile = open(filePath, 'a')

        # Write a line to the file
        myfile.write("{} {}\n".format(config.num_jobs, config.num_machines))
        
        for i in range(config.num_jobs):
            for j in range(config.num_tasksperjob):
                
                myfile.write(("{} {} ".format(machines[i,j], durations[i,j])).rstrip('\n'))
                
            myfile.write('\n')

        # Close the file
        myfile.close()
        

In [ ]:
def create_testing_dataset_cplex(config, filePath):

    counter_notFounds = 0
    
    for z in range(SIZE_TESTING_DATASET):

        start_time = time.time()
        
        machines, durations = create_random_instance(config)
        
        # Solve the problem with hard constraints
        msol, placements = placement_cplex(config, config.num_jobs, config.num_machines, machines, durations)


        if msol.is_solution():

            # Open the file with writing permission
            myfile = open(filePath, 'a')

            # Write a line to the file
            myfile.write("{} {}\n".format(config.num_jobs, config.num_machines))
            for i in range(config.num_jobs):
                for j in range(config.num_tasksperjob):

                    myfile.write(("{} {} ".format(machines[i,j], durations[i,j])).rstrip('\n'))

                myfile.write('\n')

            myfile.write("makespan: {}\n".format(msol.get_objective_values()[0]))

            # Close the file
            myfile.close()

        else:
            counter_notFounds += 1
        
        
        elapsed_time = time.time() - start_time
        
        print ("Iter: ", z, "Time: ", elapsed_time)

    print("Number of unsolved problems: ", counter_notFounds)


In [ ]:
def create_testing_dataset_ortools(config, filePath, MACHINES=None, DURATIONS=None):

    counter_notFounds = 0
    time_=[]
    
    for z in range(SIZE_TESTING_DATASET):

        start_time = time.time()
        
        machines, durations = create_random_instance(config)
        #machines = MACHINES[z]
        #durations = DURATIONS[z]

        # Solve the problem with hard constraints
        placements, makespan = placement_ortools(config, config.num_jobs, config.num_machines, machines.tolist(), durations.tolist())
        
        if makespan:

            # Open the file with writing permission
            myfile = open(filePath, 'a')

            # Write a line to the file
            myfile.write("{} {}\n".format(config.num_jobs, config.num_machines))
            for i in range(config.num_jobs):
                for j in range(config.num_tasksperjob):

                    myfile.write(("{} {} ".format(machines[i,j], durations[i,j])).rstrip('\n'))

                myfile.write('\n')

            myfile.write("makespan: {}\n".format(int(makespan)))

            # Close the file
            myfile.close()

        else:
            counter_notFounds += 1
        
        
        elapsed_time = time.time() - start_time
        time_.append(elapsed_time)
        
        print ("Iter: ", z, "Time: ", elapsed_time)
        
    print("Mean time: ", np.mean(time_))

    print("Number of unsolved problems: ", counter_notFounds)


In [ ]:
def create_testing_dataset_ortools_multiobjective(config, filePath, MACHINES=None, DURATIONS=None):

    counter_notFounds = 0
    time_=[]
    
    for z in range(SIZE_TESTING_DATASET):

        start_time = time.time()
        
        #machines, durations = create_random_instance(config)
        machines = MACHINES[z]
        durations = DURATIONS[z]

        # Solve the problem with hard constraints
        placements, makespan = placement_ortools_multiobjective(config, config.num_jobs, config.num_machines, machines.tolist(), durations.tolist())
        
        if makespan:

            # Open the file with writing permission
            myfile = open(filePath, 'a')

            # Write a line to the file
            myfile.write("{} {}\n".format(config.num_jobs, config.num_machines))
            for i in range(config.num_jobs):
                for j in range(config.num_tasksperjob):

                    myfile.write(("{} {} ".format(machines[i,j], durations[i,j])).rstrip('\n'))

                myfile.write('\n')

            myfile.write("makespan: {}\n".format(int(makespan)))

            # Close the file
            myfile.close()

        else:
            counter_notFounds += 1
        
        
        elapsed_time = time.time() - start_time
        time_.append(elapsed_time)
        
        print ("Iter: ", z, "Time: ", elapsed_time)
        
    print("Mean time: ", np.mean(time_))

    print("Number of unsolved problems: ", counter_notFounds)


In [ ]:
if __name__ == "__main__":
    
    config = Config()
    config.timeout = 1000


    
    ##########################################
    # XSmall
    ##########################################
    
    config.machine_profile = 'xsmall_default'
    config.job_profile = 'xsmall_default'
    config.reconfigure()
    
    file_test = "datasets/testing_dataset_env_xsmall_default_tout1000.data"
    machines, durations = read_instances(config, file_test)
    
    filePath = 'datasets/learning_dataset_env_xsmall_default.data'
    #create_learning_dataset(config, filePath)
    
    filePath = 'datasets/testing_dataset_env_xsmall_default_tout{}.data'.format(config.timeout)
    #create_testing_dataset(config, filePath)
    filePath = 'datasets_multiobjective/testing_dataset_env_xsmall_multiobjective2_tout{}.data'.format(config.timeout)
    #create_testing_dataset_ortools_multiobjective(config, filePath, machines, durations)
    
    ##########################################
    # Small
    ##########################################
    
    config.machine_profile = 'small_default'
    config.job_profile = 'small_default'
    config.reconfigure()
    
    file_test = "datasets/testing_dataset_env_small_default_tout1000.data"
    machines, durations = read_instances(config, file_test)
    
    filePath = 'datasets/learning_dataset_env_small_default.data'
    #create_learning_dataset(config, filePath)
    
    filePath = 'datasets/testing_dataset_env_small_default_tout{}.data'.format(config.timeout)
    #create_testing_dataset(config, filePath)
    filePath = 'datasets_multiobjective/testing_dataset_env_small_multiobjective_tout{}.data'.format(config.timeout)
    #create_testing_dataset_ortools_multiobjective(config, filePath, machines, durations)
    
    ##########################################
    # Medium
    ##########################################
    
    config.machine_profile = 'medium_default'
    config.job_profile = 'medium_default'
    config.reconfigure()

    file_test = "datasets/testing_dataset_env_medium_default_tout1000.data"
    machines, durations = read_instances(config, file_test)
    
    filePath = 'datasets/learning_dataset_env_medium_default.data'
    #create_learning_dataset(config, filePath)
    
    filePath = 'datasets/testing_dataset_env_medium_default_tout{}.data'.format(config.timeout)
    #create_testing_dataset_ortools(config, filePath, machines, durations)
    filePath = 'datasets_multiobjective/testing_dataset_env_medium_multiobjective_tout{}.data'.format(config.timeout)
    #create_testing_dataset_ortools_multiobjective(config, filePath, machines, durations)   
    
    ##########################################
    # Large
    ##########################################
    
    config.machine_profile = 'large_default'
    config.job_profile = 'large_default'
    config.reconfigure()

    file_test = "datasets/testing_dataset_env_large_default_tout3600.data"
    machines, durations = read_instances(config, file_test)
    
    filePath = 'datasets/learning_dataset_env_large_default.data'
    #create_learning_dataset(config, filePath)
    
    filePath = 'datasets/testing_dataset_env_large_default_tout{}.data'.format(config.timeout)
    #create_testing_dataset_ortools(config, filePath, machines, durations)
    filePath = 'datasets_multiobjective/testing_dataset_env_large_multiobjective_tout{}.data'.format(config.timeout)
    #create_testing_dataset_ortools_multiobjective(config, filePath, machines, durations)   
    
    
    ##########################################
    # XLarge
    ##########################################
    
    config.machine_profile = 'xlarge_default'
    config.job_profile = 'xlarge_default'
    config.reconfigure()

    file_test = "datasets/testing_dataset_env_xlarge_default_tout3600.data"
    machines, durations = read_instances(config, file_test)
    
    filePath = 'datasets/learning_dataset_env_xlarge_default.data'
    #create_learning_dataset(config, filePath)
    
    filePath = 'datasets/testing_dataset_env_xlarge_default_tout{}.data'.format(config.timeout)
    #create_testing_dataset_ortools(config, filePath)
    filePath = 'datasets_multiobjective/testing_dataset_env_xlarge_multiobjective_tout{}.data'.format(config.timeout)
    #create_testing_dataset_ortools_multiobjective(config, filePath, machines, durations)   
    